#Práctica de la Unidad 1

##**Ejercicio 1**:

**Objetivo**: Desarrollar un buscador flexible en Python que permita encontrar frases relevantes en un conjunto de textos en español, utilizando técnicas de procesamiento de lenguaje natural.

1.   Desarrollar una función `procesar_texto(texto)` que dado un texto de entrada, permita normalizar, eliminar stopwords, y lematizar.
2.   Escribir una lista de 10 frases, que utilizaremos para realizar búsquedas.
3.   Escribir una función que permita, a partir de una frase, realizar una búsqueda sobre las 10 frases, comparando con las frases procesadas con la función del punto 1. En la comparación, calcularemos un puntaje de acuerdo a las palabras en común, sin importar el orden.
4.   Mostrar un ranking descendente de las frases según el puntaje.




In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
!pip install PyPDF2

In [108]:
import re
import unicodedata
import es_core_news_sm

import PyPDF2
import pandas as pd

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
#Lista de stopwords en español.
stopwords_es = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un',
    'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le',
    'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin',
    'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo',
    'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso',
    'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo',
    'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes',
    'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo',
    'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras',
    'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya',
    'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra',
    'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es'
]

In [4]:
def remover_acentos(entrada_texto):
    nfkd_form = unicodedata.normalize('NFKD', entrada_texto)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [5]:
def eliminar_stopwords(texto):
    #Separar el texto en palabras.
    palabras = texto.split()
    # Filtrar las palabras para eliminar las stopwords.
    palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords_es]
    # Unir las palabras filtradas en una cadena y la retornamos.
    return ' '.join(palabras_filtradas)

In [6]:
#Cargar el modelo de lenguaje español
nlp = es_core_news_sm.load()

def lematizar_texto_es(texto):
    doc = nlp(texto)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return ' '.join(lemmas)

In [7]:
def procesar_texto(texto):
    #Transformar el texto a minúsculas.
    texto = texto.lower()
    #Eliminar puntuación.
    texto = re.sub(r'[^\w\s]', '', texto)
    #Eliminar acentos.
    texto = remover_acentos(texto)
    #Eliminar stopwords.
    texto = eliminar_stopwords(texto)
    #Lematizar texto.
    texto = lematizar_texto_es(texto)

    return texto

In [8]:
#Lista de frases para realizar la búsqueda.
frases = ['¡La música, sin duda, es el lenguaje universal del alma!','Después de un día agotador... ¿qué mejor que un poco de jazz relajante?',
          'Beethoven, ese genio alemán, compuso –entre otras obras maestras– la Novena Sinfonía.','Mi playlist es una locura: rock, salsa, electrónica... ¡y hasta algo de K-pop!',
          'No puedo, simplemente no puedo, concentrarme sin música instrumental de fondo.','"Esa canción", dijo entre lágrimas, "me recuerda todo lo que perdí..."',
          '¿Sabes qué es lo más curioso?”, preguntó él, mientras el piano sonaba de fondo, “esa melodía... me hace sentir como si flotara.','Toco la guitarra desde los quince años; al principio fue difícil, ahora es pasión.',
          '¡Hoy, por fin!, se lanza el nuevo álbum de mi banda favorita: ¡no puedo esperar!','Las notas altas del violín... suaves, agudas, casi etéreas... me erizan la piel.']

In [9]:
def comparar_frase(nueva_frase, lista_frases):
    puntuaciones = []
    nueva_frase_procesada = procesar_texto(nueva_frase)

    for frase in lista_frases:
        frase_procesada = procesar_texto(frase)
        puntuacion = 0
        for palabra in nueva_frase_procesada.split():
            if palabra in frase_procesada.split():
                puntuacion = puntuacion + 1
        puntuaciones.append([puntuacion, frase])

    puntuaciones = sorted(puntuaciones, key=lambda x: x[0], reverse=True)

    return puntuaciones

In [ ]:
comparaciones = comparar_frase('La melodía de esa canción, tocada en guitarra y piano al mismo tiempo, es simplemente hipnótica', frases)

for puntuacion, frase in comparaciones:
    print(f'Puntuación: {puntuacion}, Frase: {frase}')

##**Ejercicio 2**:

**Objetivo**: Desarrollar un programa en Python que permita al usuario buscar expresiones en lunfardo, basándose en descripciones de conceptos o acciones. El sistema deberá comparar la descripción del usuario con las definiciones de un diccionario de lunfardo y devolver las tres palabras cuyas definiciones sean más parecidas a la descripción ingresada.

**Preparación de Datos**:

Extraiga el texto de un diccionario de lunfardo desde un recurso en línea (enlace proporcionado), y organícelo en una tabla de dos columnas:

- Columna 1: Palabra en lunfardo.
- Columna 2: Definición de la palabra.

**Desarrollo del Programa**:

Implemente una función `buscar_en_lunfardo(descripcion)` que realice de  las siguientes tareas las que crea útiles:

- Normalización: Convertir la descripción a minúsculas y eliminar caracteres no alfanuméricos.
- Eliminación de stopwords: Remover palabras que no aporten significado relevante, utilizando una lista predefinida de stopwords en español.
- Lematización: Reducir las palabras a su forma base o lema, para facilitar la comparación con las definiciones del diccionario.
- Tokenización: Dividir la descripción en palabras individuales para su análisis.
- Comparación y Conteo:Utilizar técnicas básicas de comparación de texto para identificar las tres definiciones en el diccionario que más se asemejen a la descripción ingresada.

**Ejemplo**:

Entrada del usuario: "Persona que engaña a los demás."

Salida esperada: Podría incluir palabras como "vivo", "avivado", "pícaro", dependiendo de las definiciones disponibles en el diccionario de lunfardo.


In [ ]:
#Extraer el texto del archivo pdf: Lunfardo.pdf.
with open('Lunfardo.pdf', 'rb') as archivo:
    #Crear un objeto PdfFileReader
    lector = PyPDF2.PdfReader(archivo)

    #Inicializar una cadena vacía para almacenar el texto
    texto = ''

    #Iterar sobre todas las páginas del PDF
    for i in range(len(lector.pages)):
        #Obtener la página
        pagina = lector.pages[i]

        #Extraer el texto de la página y lo añade a la cadena de texto
        texto += pagina.extract_text ()

#Unificar el texto
texto = texto.replace('\n', ' ')

#Eliminar encabezados comunes
texto = re.sub(r'\b(LUNFARDO|DICCIONARIO[A-Z]*)\b', '', texto, flags=re.IGNORECASE)
texto = re.sub(r'\b[A-Z]\b', '', texto)  # Borra letras solas como "A", "B" que separan secciones

#Expresión regular para encontrar las entradas
regex = re.compile(r'([A-ZÁÉÍÓÚÑa-záéíóúñ() ]{2,}?)\s*:\s*(.*?)(?=(?:[A-ZÁÉÍÓÚÑa-záéíóúñ() ]{2,}?\s*:)|$)')

resultados = regex.findall(texto)

#Crear el DataFrame
dic_lunfardo = pd.DataFrame(resultados, columns=['palabra', 'definicion'])

#Limpiar texto
dic_lunfardo['palabra'] = dic_lunfardo['palabra'].str.strip()
dic_lunfardo['definicion'] = dic_lunfardo['definicion'].str.strip().str.replace(r'\s+', ' ', regex=True)

#Eliminar el texto que no corresponde al final del DataFrame.
dic_lunfardo = dic_lunfardo.iloc[:-1]
dic_lunfardo['definicion'][dic_lunfardo['palabra'] == 'Zurdo'] = 'El que milita en la izquierda política y el que profesa la ideología comunista.'

In [89]:
def buscar_en_lunfardo(descripcion):
    #Aplico funciones para procesar y comparar la frase
    descripcion = procesar_texto(descripcion)
    comparaciones = comparar_frase(descripcion, dic_lunfardo['definicion'])

    #Muestro las palabras que mas se asemejan.
    for i in range(0, 3):
        print(f"Palabra {i+1}: {df['palabra'][comparaciones[i][1] == df['definicion']].to_string(index=False)}")

In [ ]:
buscar_en_lunfardo('Persona que engaña a los demás.')

##**Ejercicio 3**:

**Objetivo**: Desarrollar un programa en Python que permita al usuario buscar expresiones en lunfardo, basándose en descripciones de conceptos o acciones. El sistema deberá comparar la descripción del usuario con las definiciones de un diccionario de lunfardo y devolver las tres palabras cuyas definiciones sean más parecidas a la descripción ingresada.

Utilice la técnica que crea más conveniente del a la unidad 2 para mejorar la performance de las respuestas. (Sólo utilice metodologías hasta word2vec).
Analice las mejoras de comportamiento del algoritmo de recomendación.



In [134]:
def buscar_en_lunfardo_tfidf(descripcion, diccionario):

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(diccionario['definicion'])

    descripcion_vectorizada = tfidf_vectorizer.transform([descripcion])

    similitudes = cosine_similarity(descripcion_vectorizada, tfidf_matrix)[0]

    top_indices = np.argsort(similitudes)[-3:][::-1]  # del más similar al menos

    print(f"Frase: {descripcion}")
    print("Top 3 palabras más parecidas:")
    for i in top_indices:
        palabra = diccionario['palabra'].iloc[i]
        similitud = similitudes[i]
        print(f"- {palabra} ({similitud:.2f})")

In [ ]:
buscar_en_lunfardo_tfidf('Persona que engaña a los demás.', dic_lunfardo)